In [2]:
import zipfile as zipfile
import pandas as pd
import datetime as dt
import folium

m = folium.Map([-5.1998, -39.2896], zoom_start=7)

print("\nLendo dados das estações pluviometricas...")

for_concat = []
with zipfile.ZipFile('../data/postos.zip', 'r') as t:
    for arq in t.namelist():
        if arq.endswith('.txt'):
            with t.open(arq) as f:
                df = pd.read_csv(f, delimiter=';')
                # Extrair o ID do nome do arquivo e adicionar como uma nova coluna
                df['id'] = arq.split('.')[0]
                for_concat.append(df)

# Exclude empty or all-NA entries before concatenation
for_concat = [
    df for df in for_concat if not df.empty and not df.isna().all().all()
]

registros_chuvas = pd.concat(for_concat)
registros_chuvas.fillna(0, inplace=True)
registros_chuvas.reset_index(drop=True, inplace=True)

# Reordenar as colunas para que 'id' seja a primeira
cols = ['id'] + [col for col in registros_chuvas.columns if col != 'id']
registros_chuvas = registros_chuvas[cols]


Lendo dados das estações pluviometricas...


In [3]:
print("\nFiltrando anos completos...")

#pegar apenas os anos completos
registros_chuvas['Anos'] = registros_chuvas['Anos'].astype(int)
registros_chuvas['Meses'] = registros_chuvas['Meses'].astype(int)
ano = dt.datetime.now().year
Anos_completos = registros_chuvas
Anos_completos = Anos_completos.loc[Anos_completos['Anos'] < ano]

Anos_completos = Anos_completos = Anos_completos.groupby('id').first().reset_index()
Anos_completos


Filtrando anos completos...


,id,Municipios,Postos,Latitude,Longitude,Anos,Meses,Total,Dia1,Dia2,...,Dia22,Dia23,Dia24,Dia25,Dia26,Dia27,Dia28,Dia29,Dia30,Dia31
0,1,Abaiara,ABAIARA,-7.361528,-39.035500,1981,1,46.2,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,7.0,1.0,11.0,0.0,0.0
1,10,Apuiarés,APUIARES,-3.950000,-39.433306,1982,1,29.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,6.5,0.0,5.0,7.0,3.5
2,100,Nova Russas,NOVA RUSSAS,-4.699528,-40.568472,1974,1,277.0,0.0,0.0,...,5.0,0.0,0.0,0.0,18.0,42.0,0.0,0.0,0.0,0.0
3,101,Novo Oriente,NOVO ORIENTE,-5.540250,-40.770417,1974,1,231.2,0.0,0.0,...,16.0,0.5,26.0,0.0,0.0,0.0,10.5,0.0,28.0,0.0
4,102,Orós,OROS,-6.243778,-38.912111,1978,1,83.5,0.0,0.0,...,0.0,0.0,0.7,0.0,0.0,4.8,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
822,95,Moraújo,MORAUJO,-3.466917,-40.677917,1981,1,40.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,34.0,1.0,0.0,0.0,5.0
823,96,Morrinhos,MORRINHOS,-3.232250,-40.129222,1984,1,38.8,0.0,10.6,...,0.0,15.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
824,97,Mucambo,MUCAMBO,-3.905694,-40.743306,1974,1,162.8,0.0,0.0,...,20.6,10.4,0.0,0.0,0.0,5.4,25.0,0.0,1.4,1.2
825,98,Mulungu,MULUNGU,-4.305722,-38.997194,1974,1,278.1,0.0,0.0,...,23.0,4.6,14.4,1.4,61.0,26.0,29.4,1.5,25.0,13.2


In [4]:
for idx, row in Anos_completos.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['Postos'],
        icon=folium.Icon(icon="cloud"),
    ).add_to(m)

In [6]:
from folium import IFrame

# Adicionar marcadores ao mapa com eventos de clique para criar/remover círculos
for idx, row in Anos_completos.iterrows():
    iframe = IFrame(f"""
    <script>
    var circleLayers = {{}};

    // Função para criar ou remover um círculo ao redor de um ponto
    function toggleCircle(lat, lng, id) {{
        if (circleLayers[id]) {{
            map.removeLayer(circleLayers[id]);
            delete circleLayers[id];
        }} else {{
            var center = [lng, lat];
            var radius = 5;
            var options = {{ steps: 10, units: "kilometers", properties: {{ foo: "bar" }} }};
            var circle = turf.circle(center, radius, options);
            circleLayers[id] = L.geoJSON(circle).addTo(map);
        }}
    }}

    // Adicionar evento de clique ao marcador
    var marker = L.marker([{row['Latitude']}, {row['Longitude']}]).addTo(map);
    marker.on('click', function() {{
        toggleCircle({row['Latitude']}, {row['Longitude']}, '{row['id']}');
    }});
    </script>
    """)
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=iframe,
        icon=folium.Icon(icon="cloud"),
    ).add_to(m)

In [7]:
m.save('../data/mapa.html')

In [8]:
m